In [1]:
!pip install timm
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from PIL import Image
import os
from transformers import EfficientNetImageProcessor, EfficientNetForImageClassification
import timm

# Check if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

import timm

model = timm.create_model("hf_hub:timm/maxvit_tiny_tf_512.in1k", pretrained=True)

model.to(device)  # Move model to GPU

# Define optimizer and scheduler
optimizer = optim.Adam(model.parameters(), lr=0.0001)
scheduler = ReduceLROnPlateau(optimizer, mode='min', patience=3, verbose=True)

# Define loss function
criterion = nn.CrossEntropyLoss()

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 26.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 19.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 44.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 MB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 13.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/586 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/124M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/torch/optim/lr_scheduler.py:28: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn("The verbose parameter is deprecated. Please use get_last_lr() "


In [4]:
!pip install torchsummary

In [5]:
from torchsummary import summary

# Assuming the model variable contains your model
summary(model, input_size=(3, 512, 512))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
        Conv2dSame-1         [-1, 64, 256, 256]           1,792
          Identity-2         [-1, 64, 256, 256]               0
          GELUTanh-3         [-1, 64, 256, 256]               0
    BatchNormAct2d-4         [-1, 64, 256, 256]             128
            Conv2d-5         [-1, 64, 256, 256]          36,928
              Stem-6         [-1, 64, 256, 256]               0
     AvgPool2dSame-7         [-1, 64, 128, 128]               0
          Identity-8         [-1, 64, 128, 128]               0
      Downsample2d-9         [-1, 64, 128, 128]               0
         Identity-10         [-1, 64, 256, 256]               0
         Identity-11         [-1, 64, 256, 256]               0
   BatchNormAct2d-12         [-1, 64, 256, 256]             128
         Identity-13         [-1, 64, 256, 256]               0
           Conv2d-14        [-1, 256, 2

In [6]:
# Initialize a counter for the number of layers
num_layers = 0

# Iterate through the model's children and count the number of layers
for child in model.children():
    num_layers += 1

print("Number of layers in the model:", num_layers)

Number of layers in the model: 4


In [7]:
from torchsummary import summary

# Define a function to select the first three layers from the model
def select_first_three_layers(model):
    first_three_layers = []
    num_layers = 0
    for name, module in model.named_children():
        first_three_layers.append(module)
        num_layers += 1
        if num_layers == 3:
            break
    return nn.Sequential(*first_three_layers)

# Select the first three layers
first_three_layers_model = select_first_three_layers(model)

# Use summary to display information for the first three layers
summary(first_three_layers_model, input_size=(3, 512, 512))


----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
        Conv2dSame-1         [-1, 64, 256, 256]           1,792
          Identity-2         [-1, 64, 256, 256]               0
          GELUTanh-3         [-1, 64, 256, 256]               0
    BatchNormAct2d-4         [-1, 64, 256, 256]             128
            Conv2d-5         [-1, 64, 256, 256]          36,928
              Stem-6         [-1, 64, 256, 256]               0
     AvgPool2dSame-7         [-1, 64, 128, 128]               0
          Identity-8         [-1, 64, 128, 128]               0
      Downsample2d-9         [-1, 64, 128, 128]               0
         Identity-10         [-1, 64, 256, 256]               0
         Identity-11         [-1, 64, 256, 256]               0
   BatchNormAct2d-12         [-1, 64, 256, 256]             128
         Identity-13         [-1, 64, 256, 256]               0
           Conv2d-14        [-1, 256, 2

In [8]:
# Initialize a counter for the number of layers
num_layers = 0

# Iterate through the model's children and count the number of layers
for child in first_three_layers_model.children():
    num_layers += 1

print("Number of layers in the model:", num_layers)

Number of layers in the model: 3


In [10]:
# Initialize a list to store parameters of the first three layers
first_three_layers_params = []

# Define a counter to track the number of layers
num_layers = 0

# Iterate through the named parameters of the model
for name, param in model.named_parameters():
    # Check if the parameter belongs to one of the first three layers
    if num_layers < 3:
        first_three_layers_params.append((name, param))

    # Increment the layer counter
    num_layers += 1

# Print the parameters of the first three layers
for name, param in first_three_layers_params:
    print(name, param.size())

stem.conv1.weight torch.Size([64, 3, 3, 3])
stem.conv1.bias torch.Size([64])
stem.norm1.weight torch.Size([64])


In [9]:
# Print the weights of the model
for name, param in first_three_layers_model.named_parameters():
    if param.requires_grad:
        print(name, param.data)

Streaming output truncated to the last 5000 lines.
         1.9457e-02, -3.0162e-01, -1.6481e-01, -1.1679e-04,  1.6805e-02,
         8.5146e-02, -2.5437e-02,  7.4611e-02,  2.0757e-01, -4.1399e-02,
         4.0484e-02, -1.6552e-01, -2.1906e-01, -7.7733e-02, -1.6064e-01,
        -1.0037e-01,  1.2057e-01,  4.8414e-03,  1.3248e-01,  1.1121e-01,
         5.2894e-02, -3.8711e-01,  2.5172e-01,  2.8113e-01,  6.8965e-02,
         1.2870e-01, -1.6378e-02, -3.9402e-01, -1.6007e-01, -1.5676e-01,
         1.7139e-01,  1.6635e-01,  1.5196e-02, -2.2267e-01, -8.0653e-02,
         7.0102e-03, -3.4135e-01, -1.1319e-01, -1.7219e-01, -8.2770e-03,
        -6.4513e-02, -1.3303e-01, -1.7455e-01,  3.6106e-02,  5.6810e-02,
        -3.9396e-02, -4.9573e-02, -2.5376e-01, -2.9527e-02,  1.3738e-01,
         1.6412e-01, -3.7698e-01,  3.9663e-02, -1.4867e-01,  1.1670e-01,
         1.0764e-01, -2.4574e-01, -2.2480e-02,  1.2622e-01,  6.2642e-02,
         9.2972e-02,  1.6252e-01, -2.2970e-01, -1.6480e-01,  9.1597e-02,
